In [ ]:
# ============================================
# Transfer Learning with MobileNetV2 for Facial Emotion Recognition
# ============================================

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report, confusion_matrix

# ------------------------------
# 1. Dataset Paths
# ------------------------------
base_dir = r"C:\Users\dilip\OneDrive\Documents\Desktop\EmoVision"

train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")
test_dir  = os.path.join(base_dir, "test")

# ------------------------------
# 2. Image Preprocessing & Augmentation
# ------------------------------
IMG_SIZE = (224, 224)   # MobileNetV2 expects 224x224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_classes = len(train_gen.class_indices)
class_labels = list(train_gen.class_indices.keys())
print("Classes:", class_labels)

# ------------------------------
# 3. Build MobileNetV2 Model
# ------------------------------
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Freeze base layers initially

# Custom head for emotion classification
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ------------------------------
# 4. Callbacks Setup
# ------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1)

callbacks = [early_stop, lr_scheduler, checkpoint]

# ------------------------------
# 5. Train the Model (Stage 1: Frozen Base)
# ------------------------------
print("\n🔹 Training top layers (base frozen)...")
history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=callbacks
)

# ------------------------------
# 6. Fine-Tune (Stage 2: Unfreeze last layers)
# ------------------------------
print("\n🔹 Fine-tuning top layers of base model...")

# Unfreeze last 30 layers of MobileNetV2
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 30
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=3e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen,
    callbacks=callbacks
)

# ------------------------------
# 7. Combine Histories for Plot
# ------------------------------
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(acc, label='Train Accuracy')
plt.plot(val_acc, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Training vs Validation Accuracy")

plt.subplot(1,2,2)
plt.plot(loss, label='Train Loss')
plt.plot(val_loss, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()

# ------------------------------
# 8. Evaluate on Test Set
# ------------------------------
print("\n🔹 Loading best saved model for evaluation...")
best_model = load_model("best_model.h5")

loss, acc = best_model.evaluate(test_gen)
print(f"\n✅ Test Accuracy: {acc*100:.2f}%")

# ------------------------------
# 9. Confusion Matrix & Classification Report
# ------------------------------
y_true = test_gen.classes
y_pred = np.argmax(best_model.predict(test_gen), axis=1)

cm = confusion_matrix(y_true, y_pred, normalize='true')
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Normalized Confusion Matrix")
plt.show()

print("\nClassification Report:\n",
      classification_report(y_true, y_pred, target_names=class_labels, zero_division=0))


Found 33846 images belonging to 6 classes.
Found 7251 images belonging to 6 classes.
Found 7259 images belonging to 6 classes.
Classes: ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,588,486 (9.87 MB)

 Trainable params: 329,990 (1.26 MB)

...

1058/1058 ━━━━━━━━━━━━━━━━━━━━ 907s 857ms/step - accuracy: 0.4206 - loss: 1.4615 - val_accuracy: 0.4686 - val_loss: 1.3801 - learning_rate: 0.0010
Epoch 4/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4274 - loss: 1.4427
Epoch 4: val_accuracy improved from 0.46863 to 0.47221, saving model to best_model.h5


1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1497s 1s/step - accuracy: 0.4258 - loss: 1.4481 - val_accuracy: 0.4722 - val_loss: 1.3687 - learning_rate: 0.0010
Epoch 5/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4348 - loss: 1.4400
Epoch 5: val_accuracy did not improve from 0.47221
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1648s 2s/step - accuracy: 0.4324 - loss: 1.4440 - val_accuracy: 0.3619 - val_loss: 2.2441 - learning_rate: 0.0010
Epoch 6/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4288 - loss: 1.4359
Epoch 6: val_accuracy did not improve from 0.47221
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1654s 2s/step - accuracy: 0.4288 - loss: 1.4409 - val_accuracy: 0.4215 - val_loss: 1.5462 - learning_rate: 0.0010
Epoch 7/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4259 - loss: 1.4418
Epoch 7: val_accuracy improved from 0.47221 to 0.47428, saving model to best_model.h5


1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1892s 2s/step - accuracy: 0.4312 - loss: 1.4354 - val_accuracy: 0.4743 - val_loss: 1.3662 - learning_rate: 0.0010
Epoch 8/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4368 - loss: 1.4271
Epoch 8: val_accuracy did not improve from 0.47428
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1853s 2s/step - accuracy: 0.4317 - loss: 1.4290 - val_accuracy: 0.4653 - val_loss: 1.3678 - learning_rate: 0.0010
Epoch 9/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4342 - loss: 1.4257
Epoch 9: val_accuracy did not improve from 0.47428
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1743s 2s/step - accuracy: 0.4362 - loss: 1.4241 - val_accuracy: 0.4696 - val_loss: 1.3682 - learning_rate: 0.0010
Epoch 10/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4498 - loss: 1.4084
Epoch 10: val_accuracy did not improve from 0.47428
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1848s 2s/step - accuracy: 0.4423 - loss: 1.4185 - val_accuracy: 0.4664 - val_loss: 1.3631 - learning_rate: 0.0010
Epoc

1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1656s 2s/step - accuracy: 0.4427 - loss: 1.4190 - val_accuracy: 0.4794 - val_loss: 1.3548 - learning_rate: 0.0010
Epoch 13/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4447 - loss: 1.4109
Epoch 13: val_accuracy improved from 0.47938 to 0.48573, saving model to best_model.h5


1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1676s 2s/step - accuracy: 0.4439 - loss: 1.4137 - val_accuracy: 0.4857 - val_loss: 1.3424 - learning_rate: 0.0010
Epoch 14/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4397 - loss: 1.4155
Epoch 14: val_accuracy did not improve from 0.48573
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1601s 2s/step - accuracy: 0.4394 - loss: 1.4175 - val_accuracy: 0.4842 - val_loss: 1.3396 - learning_rate: 0.0010
Epoch 15/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4445 - loss: 1.4053
Epoch 15: val_accuracy did not improve from 0.48573
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 1637s 2s/step - accuracy: 0.4439 - loss: 1.4110 - val_accuracy: 0.3514 - val_loss: 2.1389 - learning_rate: 0.0010
Epoch 16/20
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4422 - loss: 1.4145
Epoch 16: val_accuracy did not improve from 0.48573
1058/1058 ━━━━━━━━━━━━━━━━━━━━ 2297s 2s/step - accuracy: 0.4454 - loss: 1.4085 - val_accuracy: 0.4775 - val_loss: 1.3491 - learning_rate: 0.0010
